Experiment 2: performances of 

In [1]:
import numpy as np
from typing import Callable
from itertools import permutations
from numba import prange
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import timeit
from joblib import Parallel, delayed
from src.optimized_code import compute_risk,plurality_outcome_op,for_two_outcome_op, borda_outcome_op,veto_outcome_op, happiness_level, happiness_level_total, compute_risk_combinations, compute_risk_parallel
from src.utils import read_voting
import os
from src.outcomes import all_schemas_outcomes
from numba import prange
from src.happiness_level import HappinessLevel
import json
from src.utils import VotingSchemas, VotingArray, 
from src.strategic_voting_risk import StrategicVoting
from src.outcomes import plurality_outcome, for_two_outcome, veto_outcome, borda_outcome

/var/folders/n7/mscr753j2cv6tdsjb7n5q4kr0000gn/T/ipykernel_55322/3927595946.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
def read_voting(file_path: str, table_name: str = "voting"):
    with open(file_path) as f:
        voting = json.load(f)[table_name]
        voting = np.array(voting)

    return voting

In [8]:
path_voting = "input/voting_result.json"
if "env" in os.environ:
    if os.environ["env"] == "docker":
        path_voting = "/input/voting_result.json"
        
voting = read_voting_(path_voting, table_name="six_candidates").view(np.ndarray).view(VotingArray)



In [15]:
display(plurality_outcome_op(voting),borda_outcome_op(voting),veto_outcome_op(voting),for_two_outcome_op(voting))
display(plurality_outcome(voting).winner,borda_outcome(voting).winner,veto_outcome(voting).winner,for_two_outcome(voting).winner)


0

2

1

2

0

2

1

2

In [11]:
%timeit -n 1000 plurality_outcome(voting).winner
%timeit -n 1000 plurality_outcome_op(voting)

%timeit -n 1000 borda_outcome(voting).winner
%timeit -n 1000 borda_outcome_op(voting)

%timeit -n 1000 veto_outcome(voting).winner
%timeit -n 1000 veto_outcome_op(voting)

%timeit -n 1000 for_two_outcome(voting).winner
%timeit -n 1000 for_two_outcome_op(voting)

16.7 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.76 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
117 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
42.7 µs ± 678 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
16.4 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
30.4 µs ± 804 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
25.2 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
14.5 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
# Define the time measurements
times = [
    (16.7), (7.76),  # (mean, mean) for the first measurement pair (plurality, plurality optimized)
    (117), (42.7),   # (mean, mean) for the second measurement pair (borda, borda optimized)
    (16.4), (30.4),  # (mean, mean) for the third measurement pair (veto, veto optimized)
    (25.2),( 14.5)   # (mean, mean) for the fourth measurement pair (for two, for two optimized)
]
labels = ["Plurality", "Plurality Optimized", "Borda", "Borda Optimized", "Veto", "Veto Optimized", "Voting for Two", "Voting for Two Optimized"]
x = range(len(times))
means = [t[0] for t in times]

# Create the bar plot
plt.figure(figsize=(10, 6))
plt.bar(x, means, color='skyblue')
plt.xlabel('Voting Methods')
plt.ylabel('Time (µs)')
plt.title('Execution Time for Different Voting Methods')
plt.xticks(x, labels, rotation=45, ha='right')

# Annotate means on the bars
for i, mean in enumerate(means):
    plt.text(i, mean, f'{mean:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()



TypeError: 'float' object is not subscriptable

In [6]:
display(HappinessLevel(voting1,plurality_outcome(voting),"PLURALITY").run().total)
display(happiness_level_total(voting,plurality_outcome_op(voting)).sum())

display(HappinessLevel(voting1,for_two_outcome(voting),"FOR TWO").run().total)
display(happiness_level_total(voting,for_two_outcome_op(voting)).sum())

display(HappinessLevel(voting1,borda_outcome(voting),"BORDA").run().total)
display(happiness_level_total(voting,for_two_outcome_op(voting)).sum())

display(HappinessLevel(voting1,veto_outcome(voting),"VETO").run().total)
display(happiness_level_total(voting,veto_outcome_op(voting)).sum())

26.030240438709235

26.030240438709235

27.14349342622143

27.14349342622143

27.14349342622143

27.14349342622143

24.66849392518325

24.66849392518325

In [5]:
%timeit -n 100 (HappinessLevel(voting1,plurality_outcome(voting1),"PLURALITY").run().voter).sum()

%timeit -n 100 (HappinessLevel(voting1,for_two_outcome(voting1),"FOR TWO").run().voter).sum()

%timeit -n 100 (HappinessLevel(voting1,borda_outcome(voting1),"BORDA").run().voter).sum()

%timeit -n 100 (HappinessLevel(voting1,veto_outcome(voting1),"VETO").run().voter).sum()

%timeit -n 100 (happiness_level_total(voting,plurality_outcome_op(voting))).sum()

%timeit -n 100 (happiness_level_total(voting,for_two_outcome_op(voting))).sum()

%timeit -n 100 (happiness_level_total(voting,borda_outcome_op(voting))).sum()

%timeit -n 100 (happiness_level_total(voting,veto_outcome_op(voting))).sum()

NameError: name 'voting1' is not defined

In [ ]:
%timeit -n 5 com(voting,plurality_outcome_op)
%timeit -n 5 compute_risk(voting,for_two_outcome_op)
%timeit -n 5 compute_risk(voting,veto_outcome_op)
%timeit -n 5 compute_risk(voting,borda_outcome_op)


%timeit -n 5 StrategicVoting(voting1,HappinessLevel(voting1,plurality_outcome(voting1),"PLURALITY").run(),plurality_outcome).run().risk
%timeit -n 5 StrategicVoting(voting1,HappinessLevel(voting1,for_two_outcome(voting1),"FOR TWO").run(),for_two_outcome).run().risk
%timeit -n 5 StrategicVoting(voting1,HappinessLevel(voting1,veto_outcome(voting1),"VETO").run(),veto_outcome).run().risk
%timeit -n 5 StrategicVoting(voting1,HappinessLevel(voting1,borda_outcome(voting1),"BORDA").run(),borda_outcome).run().risk


KeyboardInterrupt: 

In [ ]:
%time display(StrategicVoting(voting1,HappinessLevel(voting1,plurality_outcome(voting1),"PLURALITY").run(),plurality_outcome).run().risk)
%time display(StrategicVoting(voting1,HappinessLevel(voting1,for_two_outcome(voting1),"FOR TWO").run(),for_two_outcome).run().risk)
%time display(StrategicVoting(voting1,HappinessLevel(voting1,veto_outcome(voting1),"VETO").run(),veto_outcome).run().risk)
%time display(StrategicVoting(voting1,HappinessLevel(voting1,borda_outcome(voting1),"BORDA").run(),borda_outcome).run().risk)



SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
compute_risk_combinations(voting3,)